# Learning Database Effectiveness Analysis

This notebook tracks the effectiveness of the learning fix database in `escalating_retry_system.py`:
- Fix reuse rates over time
- Learning efficiency metrics
- Database growth patterns
- Most reusable fix patterns
- ROI on learned fixes

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
from collections import Counter, defaultdict

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Load Fix Database

In [ ]:
# Load the learning fix database
db_path = Path('../implementation_outputs/fix_database.json')

fix_db = None
if db_path.exists():
    with open(db_path) as f:
        fix_db = json.load(f)
    print("✅ Fix database loaded successfully")
else:
    print("⚠️ Fix database not found - creating empty structure")
    fix_db = {
        'metadata': {
            'total_fixes': 0,
            'reuse_rate': 0.0,
            'learning_efficiency': 0.0
        },
        'error_signatures': {}
    }

metadata = fix_db.get('metadata', {})
error_signatures = fix_db.get('error_signatures', {})

print(f"\n📊 Database Stats:")
print(f"  Total fixes learned: {metadata.get('total_fixes', 0)}")
print(f"  Unique error signatures: {len(error_signatures)}")
print(f"  Reuse rate: {metadata.get('reuse_rate', 0):.2%}")
print(f"  Learning efficiency: {metadata.get('learning_efficiency', 0):.2%}")

## Database Growth Over Time

In [ ]:
# Analyze growth of database
fix_timeline = []

for error_sig, fixes in error_signatures.items():
    for fix in fixes:
        if 'learned_at' in fix:
            try:
                dt = datetime.fromisoformat(fix['learned_at'].replace('Z', '+00:00'))
                fix_timeline.append({
                    'timestamp': dt,
                    'error_type': fix.get('error_category', 'Unknown'),
                    'reuse_count': fix.get('reuse_count', 0),
                    'success_rate': fix.get('success_rate', 0.0)
                })
            except:
                pass

if fix_timeline:
    # Sort by timestamp
    fix_timeline.sort(key=lambda x: x['timestamp'])
    
    # Cumulative fixes learned
    cumulative_fixes = list(range(1, len(fix_timeline) + 1))
    timestamps = [f['timestamp'] for f in fix_timeline]
    
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.plot(timestamps, cumulative_fixes, linewidth=2, marker='o', markersize=4, color='green')
    ax.fill_between(timestamps, cumulative_fixes, alpha=0.3, color='green')
    ax.set_xlabel('Time')
    ax.set_ylabel('Cumulative Fixes Learned')
    ax.set_title('Learning Database Growth Over Time')
    ax.grid(alpha=0.3)
    
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    print(f"\n📈 Growth Analysis:")
    print(f"  First fix learned: {timestamps[0].strftime('%Y-%m-%d %H:%M')}")
    print(f"  Latest fix learned: {timestamps[-1].strftime('%Y-%m-%d %H:%M')}")
    print(f"  Total fixes in database: {len(fix_timeline)}")
    
    # Calculate learning rate
    time_span = (timestamps[-1] - timestamps[0]).total_seconds() / 3600  # hours
    if time_span > 0:
        learning_rate = len(fix_timeline) / time_span
        print(f"  Learning rate: {learning_rate:.2f} fixes/hour")
else:
    print("No timeline data available yet - database is empty")

## Fix Reuse Analysis

In [ ]:
# Analyze which fixes are most reused
reuse_data = []

for error_sig, fixes in error_signatures.items():
    for i, fix in enumerate(fixes):
        reuse_count = fix.get('reuse_count', 0)
        success_rate = fix.get('success_rate', 0.0)
        error_type = fix.get('error_category', 'Unknown')
        
        reuse_data.append({
            'error_signature': error_sig[:50] + '...' if len(error_sig) > 50 else error_sig,
            'fix_index': i,
            'reuse_count': reuse_count,
            'success_rate': success_rate,
            'error_type': error_type,
            'roi': reuse_count * success_rate  # Simple ROI metric
        })

if reuse_data:
    # Sort by reuse count
    reuse_data.sort(key=lambda x: x['reuse_count'], reverse=True)
    
    # Plot top 10 most reused fixes
    top_10 = reuse_data[:10]
    
    if top_10:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        
        # Reuse counts
        labels = [f"Fix {i+1}" for i in range(len(top_10))]
        counts = [f['reuse_count'] for f in top_10]
        ax1.barh(labels, counts, color='teal')
        ax1.set_xlabel('Reuse Count')
        ax1.set_title('Top 10 Most Reused Fixes')
        ax1.invert_yaxis()
        
        # Success rates
        success_rates = [f['success_rate'] * 100 for f in top_10]
        ax2.barh(labels, success_rates, color='orange')
        ax2.set_xlabel('Success Rate (%)')
        ax2.set_title('Success Rate of Top Reused Fixes')
        ax2.invert_yaxis()
        
        plt.tight_layout()
        plt.show()
    
    print("\n🏆 Most Effective Fixes:")
    # Sort by ROI
    reuse_data.sort(key=lambda x: x['roi'], reverse=True)
    for i, fix in enumerate(reuse_data[:5], 1):
        print(f"\n{i}. Error Type: {fix['error_type']}")
        print(f"   Reuse Count: {fix['reuse_count']}")
        print(f"   Success Rate: {fix['success_rate']:.1%}")
        print(f"   ROI Score: {fix['roi']:.2f}")
    
    # Calculate overall reuse statistics
    total_reuses = sum(f['reuse_count'] for f in reuse_data)
    avg_reuse = total_reuses / len(reuse_data) if reuse_data else 0
    
    print(f"\n📊 Reuse Statistics:")
    print(f"  Total fix applications: {total_reuses}")
    print(f"  Average reuse per fix: {avg_reuse:.2f}")
    print(f"  Fixes never reused: {sum(1 for f in reuse_data if f['reuse_count'] == 0)}")
    print(f"  Fixes reused 5+ times: {sum(1 for f in reuse_data if f['reuse_count'] >= 5)}")
else:
    print("No reuse data available - database hasn't been utilized yet")

## Error Type Coverage

In [ ]:
# Analyze coverage of different error types
error_type_coverage = Counter()

for error_sig, fixes in error_signatures.items():
    for fix in fixes:
        error_type = fix.get('error_category', 'Unknown')
        error_type_coverage[error_type] += 1

if error_type_coverage:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    types = list(error_type_coverage.keys())
    counts = list(error_type_coverage.values())
    
    colors = plt.cm.Set3(range(len(types)))
    ax.pie(counts, labels=types, autopct='%1.1f%%', colors=colors, startangle=90)
    ax.set_title('Fix Coverage by Error Type')
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Coverage by Error Type:")
    for error_type, count in error_type_coverage.most_common():
        print(f"  {error_type:20s}: {count:3d} fixes available")
else:
    print("No error type data available")

## Learning Efficiency Metrics

In [ ]:
# Calculate advanced efficiency metrics
if reuse_data:
    # Time saved calculation (assume 5 minutes per manual fix)
    total_reuses = sum(f['reuse_count'] for f in reuse_data)
    time_saved_minutes = total_reuses * 5
    time_saved_hours = time_saved_minutes / 60
    
    # Success-weighted efficiency
    successful_reuses = sum(f['reuse_count'] * f['success_rate'] for f in reuse_data)
    
    # Database utilization
    utilized_fixes = sum(1 for f in reuse_data if f['reuse_count'] > 0)
    utilization_rate = utilized_fixes / len(reuse_data) if reuse_data else 0
    
    print("\n⚡ Efficiency Metrics:")
    print(f"  Total fix applications: {total_reuses}")
    print(f"  Successful applications: {successful_reuses:.1f}")
    print(f"  Estimated time saved: {time_saved_hours:.1f} hours")
    print(f"  Database utilization: {utilization_rate:.1%}")
    print(f"  Average success rate: {(sum(f['success_rate'] for f in reuse_data) / len(reuse_data)):.1%}")
    
    # ROI visualization
    fig, ax = plt.subplots(figsize=(10, 5))
    
    # Plot ROI distribution
    roi_values = [f['roi'] for f in reuse_data if f['roi'] > 0]
    if roi_values:
        ax.hist(roi_values, bins=20, color='green', alpha=0.7, edgecolor='black')
        ax.set_xlabel('ROI Score (Reuse Count × Success Rate)')
        ax.set_ylabel('Number of Fixes')
        ax.set_title('Distribution of Fix ROI')
        ax.axvline(sum(roi_values) / len(roi_values), color='red', linestyle='--', 
                   label=f'Average ROI: {sum(roi_values) / len(roi_values):.2f}')
        ax.legend()
        ax.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
else:
    print("Not enough data for efficiency metrics")

## Recommendations

In [ ]:
print("\n🎯 DATABASE OPTIMIZATION RECOMMENDATIONS:\n")

if reuse_data:
    # Find underutilized fixes
    never_used = sum(1 for f in reuse_data if f['reuse_count'] == 0)
    if never_used > 0:
        print(f"1. {never_used} fixes have never been reused - consider pruning or improving signatures")
    
    # Find low success rate fixes
    low_success = [f for f in reuse_data if f['success_rate'] < 0.5 and f['reuse_count'] > 0]
    if low_success:
        print(f"2. {len(low_success)} fixes have <50% success rate - review and improve these")
    
    # Coverage gaps
    if error_type_coverage:
        min_coverage = min(error_type_coverage.values())
        min_type = [k for k, v in error_type_coverage.items() if v == min_coverage][0]
        print(f"3. '{min_type}' has lowest coverage ({min_coverage} fixes) - add more patterns")
    
    # High performers
    high_roi = [f for f in reuse_data if f['roi'] > 10]
    if high_roi:
        print(f"4. {len(high_roi)} fixes have high ROI - document and replicate their patterns")
    
    print("\n5. Continue learning - database is actively improving the system")
    print("6. Monitor reuse rate trends to validate learning effectiveness")
else:
    print("1. Database is empty - start running retry system to build knowledge")
    print("2. Ensure escalating_retry_system.py is properly configured")
    print("3. Allow system to learn from at least 10 successful fixes before analysis")